# Point Cloud Diffusion 

## Imports

In [ ]:
# Dataset-related
from pclab.datasets.modelnet import ModelNet40
from pclab.transforms import *
from pclab.learner import *
from torch.utils.data import DataLoader

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim

# Visualization
from pclab.utils import quick_vis

# Other
from functools import partial
from pclab.models.pointnet import LinearLayer

# Lightning
import pytorch_lightning as pl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2023-11-29 11:34:48,958 - instantiator - Created a temporary directory at /tmp/tmpokpiiryu
INFO - 2023-11-29 11:34:48,959 - instantiator - Writing /tmp/tmpokpiiryu/_remote_module_non_scriptable.py


## Dataset

In [ ]:
path = "/home/ioannis/Desktop/programming/data"

transforms = [RandomPointKeep(1024), UnitSphereNormalization(), AnisotropicScale(), ToTensor()]

train_dataset = ModelNet40(path, 2048, 'train', transforms=transforms, category=0)
valid_dataset = ModelNet40(path, 2048, 'test' , transforms=transforms, category=0)

train_dl, valid_dl = map(partial(DataLoader, batch_size=32, shuffle=True, num_workers=8, drop_last=True), (train_dataset, valid_dataset))
len(train_dl)

19

## Point AutoEncoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.ln1 = LinearLayer(in_features, out_features)
        
    def forward(self, pc): # expecting `pc` to be of shape `BxFxN`, where `F` is the number of input features and `N` is the number of points in the point cloud
        return self.ln1(pc)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.ln1 = LinearLayer(in_features, out_features)
    
    def forward(self, pc): # expecting `pc` to be of shape `BxFxN`, where `F` is the number of input features and `N` is the number of points in the point cloud
        return self.ln1(pc)

In [ ]:
class PointAutoEncoder(pl.LightningModule):
    
    def __init__(self, point_dim, latent_dim, noise_scale=0.2):
        super().__init__()
        self.point_dim, self.latent_dim = point_dim, latent_dim
        self.noise_scale = noise_scale
        
        self.encoder = Encoder(point_dim, latent_dim)
        self.decoder = Decoder(latent_dim, point_dim)
        
        self.loss_fn = nn.MSELoss()
        
    def forward(self, pc):
        pass
    
    def training_step(self, batch, batch_idx):
        
        pc, _ = batch
        
        # add noise on the point clouds to move points to new locations
        noise = self.noise_scale * torch.randn_line(pc).to(pc.device)
        pc = pc + noise
        
        # encoding points to latent space
        latent_repr = self.encoder(pc)
        
        # decoding points back to point space
        pc_rec = self.decoder(latent_repr)
        
        # MSE loss
        loss = self.loss_fn(pc_rec, pc)
        
        return loss
        
    def validation_step(self, batch, batch_idx):
        
        pc, _ = batch
        
        # add noise on the point clouds to move points to new locations
        noise = self.noise_scale * torch.randn_line(pc).to(pc.device)
        pc = pc + noise
        
        # encoding points to latent space
        latent_repr = self.encoder(pc)
        
        # decoding points back to point space
        pc_rec = self.decoder(latent_repr)
        
        # MSE loss
        loss = self.loss_fn(pc_rec, pc)
    
    def configure_optimizers(self):
        opt = optim.Adam(params=self.parameters(), lr=1e-3, weight_decay=0.05)
        return opt

### Train the PointAutoEncoder

In [ ]:
model = PointAutoEncoder(point_dim=3, latent_dim=64)

trainer = pl.Trainer(
    accelerator='gpu', 
    auto_lr_find=True)

tr


/home/ioannis/anaconda3/envs/vvrenv/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


MisconfigurationException: No supported gpu backend found!

In [ ]:
trainer = pl.Trainer(
    accelerator='gpu',
    devices=1,
    max_epochs=5
)

In [ ]:
torch.cuda.is_available()

True